### DAY 9 – TASK: Cleaning

***Today's Goal:*** Make the delivery_time_days column clean and usable.

### Load data

In [1]:
# load datasets 
import pandas as pd

orders = pd.read_csv("DATA/olist_orders_dataset.csv")
customers = pd.read_csv("DATA/olist_customers_dataset.csv")

merged_df = pd.merge(
    orders,
    customers,
    on='customer_id',
    how='left'
)

# convert datatyoe object into datetime 
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    merged_df[col] = pd.to_datetime(merged_df[col],errors='coerce')

# Create delivery time feature
merged_df['delivery_time_days'] = (
    merged_df['order_delivered_customer_date'] - # date when the customer received the order
    merged_df['order_purchase_timestamp'] # date when the customer placed the order
).dt.days

### Check invalid values
- Find how many values are:
    - Negative
    - Too large (example: more than 60 days)

In [2]:
(merged_df['delivery_time_days'] < 0).sum() # check negative values 

np.int64(0)

Too large values

In [3]:
mean = merged_df['delivery_time_days'].mean() # Find average delivery time 
std = merged_df['delivery_time_days'].std() # Find how much delivery times vary
too_large_threshold = mean + 3*std # Anything way above normal (mean + 3*std) → “too large”
too_large_values = (merged_df['delivery_time_days'] > too_large_threshold).sum() # Count how many orders are too long
too_large_values

np.int64(1614)

In [4]:
merged_df['order_status'].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

### Apply Business Logic 
- Keep delivery_time_days only if order_status = delivered
- Otherwise set to NaN
- Only delivered orders should have non-NaN delivery times

In [5]:
import numpy as np
merged_df.loc[
    merged_df['order_status'] != 'delivered',
    'clean_delivery_time_days'
] = np.nan

In [6]:
merged_df.groupby('order_status')['delivery_time_days'].count()

order_status
approved           0
canceled           6
created            0
delivered      96470
invoiced           0
processing         0
shipped            0
unavailable        0
Name: delivery_time_days, dtype: int64

#### Create Clean Delivery Time Column

In [7]:
merged_df['clean_delivery_time_days'] = merged_df['delivery_time_days']

#### Select Valid Delivered Order
- We select only delivered orders with valid delivery times to calculate correct statistics and handle outliers properly
- We ignore all non-delivered or missing orders because their delivery time is not valid.

In [8]:
valid_delivery = merged_df[
    (merged_df['order_status'] == 'delivered') &
    (merged_df['delivery_time_days'].notna())
]['delivery_time_days']

valid_delivery.shape


(96470,)

In [9]:
valid_delivery.count()

np.int64(96470)

### Handle extreme values

In [10]:
mean = valid_delivery.mean()
std = valid_delivery.std()

mean, std

(np.float64(12.093604229294082), np.float64(9.551379928203627))

In [11]:
upper_limit = mean + 3 * std
upper_limit

np.float64(40.74774401390496)

***We use mean + 3×std to find the maximum normal delivery time; 
anything above this is treated as an outlier.***

In [12]:
merged_df.loc[
    merged_df['order_status'] != 'delivered',
    'clean_delivery_time_days'
] = pd.NA


##### Handle Negative Delivery Times
- Replace Negative Delivery Days with NaN
- Clean Invalid Negative Values

In [13]:
merged_df.loc[
    merged_df['clean_delivery_time_days'] < 0,
    'clean_delivery_time_days'
] = pd.NA

##### Handle Extreme Delivery Times
- Mark Outliers as NaN"
- Remove Unusually Long Delivery Days

In [14]:
merged_df.loc[
    merged_df['clean_delivery_time_days'] > upper_limit,
    'clean_delivery_time_days'
] = pd.NA

In [15]:
merged_df.groupby('order_status')['clean_delivery_time_days'].count() 
# Check Cleaned Delivery Times by Order Status

order_status
approved           0
canceled           0
created            0
delivered      94856
invoiced           0
processing         0
shipped            0
unavailable        0
Name: clean_delivery_time_days, dtype: int64

#### Observations 
- Most orders are delivered in a normal number of days.
- There are no negative delivery times in the dataset.
- Orders that are canceled, shipped, or processing should not be counted.
- Very long delivery times are rare and considered outliers.

#### Learnings

- Always preserve raw data; work on a separate clean column.
- Apply business logic before statistical cleaning.
- Outlier handling should be data-driven, not fixed.
- Correct order of operations is critical for reliable results.